### import yelp databases, clean, organize and merge into final databases.

In [69]:
import pandas as pd

In [70]:
#import data csv
spa_data = pd.read_csv('finalspadata.csv')
rest_data = pd.read_csv('finaldata.csv')  
hotel_data = pd.read_csv('finalhoteldata.csv')

In [71]:
#convert price to numeric value
def cleandata(data):
    data = data.drop(columns={'Unnamed: 0', 'BusinessType'})
    data['price']=data['price'].apply(lambda x: 1 if (x == "€") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "€€") else x)
    data['price']=data['price'].apply(lambda x: 3 if (x == "€€€") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "££") else x)
    data['price']=data['price'].apply(lambda x: 1 if (x == "$") else x)
    data['price']=data['price'].apply(lambda x: 2 if (x == "$$") else x)
    data['price']=data['price'].apply(lambda x: 3 if (x == "$$$") else x)
    data['price']=data['price'].apply(lambda x: 4 if (x == "$$$$") else x)
    data['price'] = data['price'].astype('int')
    data['City'] = data['City'].apply(lambda x: x.strip())
    data['State'] = data['State'].apply(lambda x: x.strip())
    
    #drop 0 values from database
    data = data.loc[data['price'] != 0]
    
    return data


In [72]:
#call function to clean data
spa_data = cleandata(spa_data)
rest_data = cleandata(rest_data)
hotel_data = cleandata(hotel_data)

In [73]:
rest_data

,State,City,name,cat,price,rating,review,Total
1,al,butler,Don Rafa,Mexican,2,5.0,4,6
5,al,butler,McDonald's,Coffee & Tea,1,1.0,3,6
8,al,cherokee,Point Of View Restaurant And Oyster Bar,Seafood,2,4.0,2,8
9,al,cherokee,Rattlesnake Saloon,Bars,1,3.5,50,8
10,al,cherokee,The Southern Farm Table Restaurant and Bakery,American (Traditional),2,4.5,14,8
...,...,...,...,...,...,...,...,...
65751,id,boise,Jimmy John's,Sandwiches,1,3.0,21,931
65752,id,boise,Wingstop,Chicken Wings,1,3.0,60,931
65753,id,boise,Fish Poké Bar,Poke,2,4.0,33,931
65754,id,boise,Louie's Pizza and Italian Restaurant,Pizza,2,3.5,114,931


In [74]:
new_york = rest_data.loc[rest_data['City'] =='new york']

In [75]:
#make a seperate database of expensive and cheap for business type for seperate analysis
exp_spa_data = spa_data.loc[spa_data['price'] >= 3]
cheap_spa_data = spa_data.loc[spa_data['price'] < 3]
exp_rest_data = rest_data.loc[rest_data['price'] >= 3]
cheap_rest_data = rest_data.loc[rest_data['price'] < 3]
exp_hotel_data = hotel_data.loc[hotel_data['price'] >= 3]
cheap_hotel_data = hotel_data.loc[hotel_data['price'] < 3] 

In [76]:
#read in PCI and pop data, clean and sort
other_db = pd.read_csv("final_before_yelp.csv")
other_db = other_db.drop(columns={'Unnamed: 0'})
other_db = other_db.rename(columns={'DataValue':'PCI'})
other_db = other_db.sort_values('State')
other_db.reset_index(drop=True, inplace=True)
other_db['City'] = other_db['City'].apply(lambda x: x.lower())
other_db['State'] = other_db['State'].apply(lambda x: x.lower())
other_db['PCI'] = other_db['PCI'].apply(lambda x: x.replace(',', ''))
other_db['PCI'] = other_db['PCI'].astype('int')


In [77]:
new_york_merge = other_db.merge(new_york, on=['State', 'City'])

In [78]:
new_york_merge.drop(['State', 'City', 'name', 'Total'], axis=1, inplace=True)

In [79]:
new_york_merge.to_csv('new_york_merge.csv', index=False)

In [80]:
#function that creates a count of each price point and merges all databases

def buscount(data):
    
    data_means = data.groupby(['State', 'City']).mean()
#     count = data.groupby(['State', 'City']).count()['name']
#     data_count = pd.DataFrame(count)
#     data_count.rename(columns={'name':'P total'}, inplace=True)
#     data1 = data_count.merge(data_means, on=['State', 'City'], how='inner')
    data1['Price perc'] = data1['P total'] / data1['Total']    
    data1.reset_index(inplace=True)
    data1.drop(['Total', 'P total'], axis=1, inplace=True)
    
    final = other_db.merge(data1, how='inner', on=['State', 'City'])
    return final

In [44]:
#function that process for full data
def fullcount(data):
    
    data_means = data.groupby(['State', 'City']).mean()
#     count = data.groupby(['State', 'City']).count()['name']
#     data_count = pd.DataFrame(count)
#     data_count.rename(columns={'name':'P total'}, inplace=True)
#     data1 = data_count.merge(data_means, on=['State', 'City'], how='inner')
#     data1.reset_index(inplace=True)
    final = other_db.merge(data_means, how='inner', on=['State', 'City'])
    final['PP'] = final['Total'] / final['Pop']    

#     final.drop(['Total', 'P total'], axis=1, inplace=True)
    
    return final

In [82]:
#run databases through buscount() in order to aggregate db
exp_spa_data = buscount(exp_spa_data)
cheap_spa_data = buscount(cheap_spa_data)
exp_rest_data = buscount(exp_rest_data)
cheap_rest_data = buscount(cheap_rest_data)
exp_hotel_data = buscount(exp_hotel_data)
cheap_hotel_data= buscount(cheap_hotel_data)
full_hotel_data = fullcount(hotel_data)
full_rest_data = fullcount(rest_data)

In [84]:
full_hotel = full_hotel_data.merge(full_rest_data, on=['State', 'City'])

In [45]:
full_rest_data = fullcount(rest_data)

In [47]:
full_rest_data.to_csv("Resources/full_rest_data_new.csv")


In [29]:
test_count = rest_data.groupby(['State', 'City']).count()['name']

In [30]:
t = pd.DataFrame(test_count)

In [35]:
t

name
State City                              
ak    anchorage municipality         202
      bristol bay borough              2
      fairbanks north star borough     1
      haines borough                   7
      juneau city and borough         34
...                                  ...
wv    mason                           12
      morgan                          11
      wayne                            2
wy    laramie                         30
      sheridan                        15

[1213 rows x 1 columns]

In [37]:
t.head(50)

name
State City                              
ak    anchorage municipality         202
      bristol bay borough              2
      fairbanks north star borough     1
      haines borough                   7
      juneau city and borough         34
      kenai peninsula borough         18
      matanuska-susitna borough       19
      sitka city and borough          15
al    butler                           2
      cherokee                         5
      clay                           106
      cullman                         29
      elmore                          32
      fayette                          7
      franklin                         9
      geneva                           5
      jackson                         16
      madison                        199
      marion                           6
      mobile                         175
      talladega                       22
      tuscaloosa                     105
ar    benton                          22
      bradley                          8
      conway                          42
      franklin                         2
      fulton                           3
      garland                         34
      johnson                         48
      lincoln                          6
      lonoke                          12
      madison                          9
      marion                         114
      perry                            4
      searcy                          29
      st. francis                      4
      van buren                       26
      washington                      59
az    maricopa                        17
      pima                            10
ca    alameda                        212
      colusa                          18
      fresno                         208
      imperial                        15
      los angeles                    224
      madera                          48
      merced                          52
      monterey                       163
      napa                           152
      orange                         225

In [86]:
#save databases to csv
exp_spa_data.to_csv('exp_spa_data.csv', index=False)
cheap_spa_data.to_csv('cheap_spa_data.csv', index=False)
exp_rest_data.to_csv('exp_rest_data.csv', index=False)
cheap_rest_data.to_csv('cheap_rest_data.csv', index=False)
exp_hotel_data.to_csv('exp_hotel_data.csv', index=False) 
cheap_hotel_data.to_csv('cheap_hotel_data.csv', index=False)
full_hotel.to_csv('full_hotel_data.csv', index=False)

In [36]:
#seperate database into high Population and low population database
high_pop = final_db[(final_db['Pop'] > 60000)]
low_pop = final_db[(final_db['Pop'] < 60000)]


In [61]:
high_pop = high_pop.merge(data1_means, how='inner', on=['State', 'City'])
low_pop = low_pop.merge(data1_means, how='inner', on=['State', 'City'])

In [64]:
hp_f = final_db.merge(hp_final, how='inner', on=['State', 'City'])

In [252]:
final_final = final_db.merge(data1_means, how='inner', on=['State', 'City'])

In [62]:
high_pop.to_csv('high_pop.csv')
low_pop.to_csv('low_pop.csv')